#  Exercício 1 - Tratamento de Dados com PySpark

## Objetivo

# Demonstrar técnicas de limpeza e preparação de dados usando PySpark

## Dataset

- videos-stats.csv: Estatísticas de vídeos
- comments.csv: Comentários dos vídeos
- USvideos.csv: Vídeos dos EUA (para join)

## Técnicas aplicadas:

1. Tratamento de valores nulos
2. Conversão de tipos de dados
3. Remoção de duplicatas
4. Criação de novas colunas
5. Junção de DataFrames
6. Exportação para formato Parquet

In [ ]:
# Instalação do PySpark
!pip install pyspark

In [ ]:
# Importação de Bibliotecas
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, regexp_extract, sum as spark_sum, to_date, date_format

In [ ]:
# Inicialização da Sessão Spark
spark = SparkSession.builder.appName("EBAC_Data_Cleaning").getOrCreate()

In [ ]:
# Leitura do Arquivo videos-stats.csv
df_video = spark.read \
    .option('header', 'true') \
    .option("inferSchema", "true") \
    .csv('/content/drive/MyDrive/Material de apoio - M27 (1)/videos-stats.csv')

In [ ]:
print("Schema do DataFrame de vídeos:")
df_video.printSchema()

print("\nPrimeiras 10 linhas:")
df_video.show(10)

In [ ]:
# Análise inicial dos dados
print("Quantidade de registros em df_video:", df_video.count())
print("Colunas disponíveis:", df_video.columns)

In [ ]:
# Tratamento de Valores Nulos - Substituição por 0
print("Valores nulos antes do tratamento:")
null_counts_before = df_video.agg(*[
    spark_sum(col(c).isNull().cast("int")).alias(c)
    for c in ['Likes', 'Comments', 'Views']
])

null_counts_before.show()

In [ ]:
# Substitui valores nulos nas colunas numéricas por 0
df_video = df_video.na.fill({
    'Likes': 0,
    'Comments': 0,
    'Views': 0
})

print("Valores nulos após o tratamento:")
null_counts_after = df_video.agg(*[
    spark_sum(col(c).isNull().cast("int")).alias(c)
    for c in ['Likes', 'Comments', 'Views']
])

null_counts_after.show()

In [ ]:
# Leitura do Arquivo comments.csv
df_comentario = spark.read \
    .option('header', 'true') \
    .option("inferSchema", "true") \
    .csv('/content/drive/MyDrive/Material de apoio - M27 (1)/comments.csv')

print("Schema do DataFrame de comentários:")
df_comentario.printSchema()

print("\nPrimeiras 10 linhas de comentários:")
df_comentario.show(10)

In [ ]:
# Análise dos dados de comentários
print("Quantidade de registros em df_comentario:", df_comentario.count())

In [ ]:
# Remove linhas com Video ID nulo
df_video_sem_nulos = df_video.dropna(subset=["Video ID"])
df_comentario_sem_nulos = df_comentario.dropna(subset=["Video ID"])

print("Quantidade após remoção de nulos em Video ID:")
print(f"Vídeos: {df_video_sem_nulos.count()}")
print(f"Comentários: {df_comentario_sem_nulos.count()}")

In [ ]:
# Remove duplicatas baseadas no Video ID
df_video = df_video.dropDuplicates(subset=['Video ID'])
print("Quantidade após remoção de duplicatas:", df_video.count())

In [ ]:
# Conversão de tipos de dados
df_video = df_video.withColumn("Likes", col("Likes").cast("int")) \
                   .withColumn("Comments", col("Comments").cast("int")) \
                   .withColumn("Views", col("Views").cast("int"))

df_comentario = df_comentario.withColumn("Likes", col("Likes").cast("int")) \
                             .withColumn("Sentiment", col("Sentiment").cast("int")) \
                             .withColumnRenamed('Likes', 'Likes_Comment')

print("Schema após conversão de tipos:")
df_video.printSchema()

In [ ]:
# Criação de nova coluna: Interaction (soma de Likes, Comments e Views)
df_video = df_video.withColumn('Interaction', col('Likes') + col('Comments') + col('Views'))
df_video.select('Video ID', 'Title', 'Likes', 'Comments', 'Views', 'Interaction').show(10)

In [ ]:
# Transformação de datas
df_video = df_video.withColumn("Published_At", to_date(col("Published At"))) \
                   .withColumn('Year', date_format(col('Published_At'), 'yyyy'))

print("Datas transformadas:")
df_video.select('Published At', 'Published_At', 'Year').show(10)

In [ ]:
# JOIN entre vídeos e comentários
df_join_video_comments = df_video.join(df_comentario, "Video ID", "inner")
print(f"Quantidade de registros após JOIN: {df_join_video_comments.count()}")
df_join_video_comments.show(10)

In [ ]:
# Leitura e JOIN com dados dos EUA
df_us_videos = spark.read \
    .option('header', 'true') \
    .option("inferSchema", "true") \
    .csv('/content/drive/MyDrive/Material de apoio - M27 (1)/USvideos.csv')

df_join_video_usvideos = df_video.join(df_us_videos, "Title", "inner")
print(f"Quantidade de registros após JOIN com US videos: {df_join_video_usvideos.count()}")
df_join_video_usvideos.show(10)

In [ ]:
# Remoção de colunas desnecessárias
colunas_para_remover = ['_c0']
for coluna in colunas_para_remover:
    if coluna in df_video.columns:
        df_video = df_video.drop(coluna)
        print(f"Coluna '{coluna}' removida do DataFrame de vídeos")

    if coluna in df_join_video_comments.columns:
        df_join_video_comments = df_join_video_comments.drop(coluna)
        print(f"Coluna '{coluna}' removida do DataFrame de join")

In [ ]:
# Exportação para Parquet (formato otimizado)
df_video.write.mode('overwrite').option('header', 'true') \
          .parquet('/content/drive/MyDrive/Projeto Pyspark/videos-tratados-parquet')

df_join_video_comments.write.mode('overwrite').option('header', 'true') \
          .parquet('/content/drive/MyDrive/Projeto Pyspark/videos-comments-tratados-parquet')

print("Dados exportados para Parquet com sucesso!")

In [ ]:
# Verificação dos dados salvos
print("Dados de vídeos salvos:")
spark.read.option('header', 'true') \
    .parquet('/content/drive/MyDrive/Projeto Pyspark/videos-tratados-parquet') \
    .show(5)

print("Dados de vídeos e comentários salvos:")
spark.read.option('header', 'true') \
    .parquet('/content/drive/MyDrive/Projeto Pyspark/videos-comments-tratados-parquet') \
    .show(5)

In [ ]:
# Encerramento da Sessão Spark
spark.stop()
print("Sessão Spark encerrada.")